In [1]:
#임베딩
sentences = ['nice great best amazing', 'stop lies', 'pitiful nerd', 'excellent work', 'supreme quality', 'bad', 'highly respectable']
y_train = [1, 0, 0, 1, 1, 0, 1]

In [2]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [5]:
tkz = Tokenizer()
tkz.fit_on_texts(sentences)
vocab_size = len(tkz.word_index)+1
print('단어 집합 크기 : ', vocab_size)

단어 집합 크기 :  16


In [9]:
x_encoded = tkz.texts_to_sequences(sentences)
max_len = max(len(l) for l in x_encoded)
print('단어 최대 길이 : ', max_len)

단어 최대 길이 :  4


In [10]:
x_train = pad_sequences(x_encoded, maxlen = max_len, padding = 'post')
y_train = np.asarray(y_train)
print('패딩 결과 :')
print(x_train)

패딩 결과 :
[[ 1  2  3  4]
 [ 5  6  0  0]
 [ 7  8  0  0]
 [ 9 10  0  0]
 [11 12  0  0]
 [13  0  0  0]
 [14 15  0  0]]


In [45]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Flatten, Input


In [11]:

embedding_dim = 4

model = Sequential()
e = Embedding(vocab_size, embedding_dim, input_length = max_len)
model.add(e)
model.add(Flatten())
model.add(Dense(1, activation = 'sigmoid'))
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['acc'])
model.fit(x_train, y_train, epochs = 100, verbose = 2)

Epoch 1/100
1/1 - 1s - loss: 0.6876 - acc: 0.7143 - 538ms/epoch - 538ms/step
Epoch 2/100
1/1 - 0s - loss: 0.6862 - acc: 0.7143 - 16ms/epoch - 16ms/step
Epoch 3/100
1/1 - 0s - loss: 0.6848 - acc: 0.8571 - 0s/epoch - 0s/step
Epoch 4/100
1/1 - 0s - loss: 0.6833 - acc: 1.0000 - 0s/epoch - 0s/step
Epoch 5/100
1/1 - 0s - loss: 0.6819 - acc: 1.0000 - 3ms/epoch - 3ms/step
Epoch 6/100
1/1 - 0s - loss: 0.6805 - acc: 1.0000 - 0s/epoch - 0s/step
Epoch 7/100
1/1 - 0s - loss: 0.6791 - acc: 1.0000 - 0s/epoch - 0s/step
Epoch 8/100
1/1 - 0s - loss: 0.6776 - acc: 1.0000 - 16ms/epoch - 16ms/step
Epoch 9/100
1/1 - 0s - loss: 0.6762 - acc: 1.0000 - 0s/epoch - 0s/step
Epoch 10/100
1/1 - 0s - loss: 0.6748 - acc: 1.0000 - 0s/epoch - 0s/step
Epoch 11/100
1/1 - 0s - loss: 0.6733 - acc: 1.0000 - 16ms/epoch - 16ms/step
Epoch 12/100
1/1 - 0s - loss: 0.6719 - acc: 1.0000 - 0s/epoch - 0s/step
Epoch 13/100
1/1 - 0s - loss: 0.6704 - acc: 1.0000 - 0s/epoch - 0s/step
Epoch 14/100
1/1 - 0s - loss: 0.6690 - acc: 1.0000 - 

In [13]:
# 외부 Glove 데이터 활용

from urllib.request import urlretrieve, urlopen
import gzip
import zipfile

In [16]:
urlretrieve("http://nlp.stanford.edu/data/glove.6B.zip", filename="glove.6B.zip")
zf=zipfile.ZipFile('glove.6B.zip')
zf.extractall()
zf.close()

In [22]:
embedding_dict = dict()

f = open('glove.6B.100d.txt', encoding="utf8")

for line in f:
    word_vector = line.split()
    word = word_vector[0]
    
    word_vector_arr = np.asarray(word_vector[1:], dtype='float32')
    embedding_dict[word] = word_vector_arr
f.close()

In [23]:
print('Embedding vector : %s 개' % len(embedding_dict))

Embedding vector : 400000 개


In [24]:
print(embedding_dict['respectable'])
print('벡터의 차원 수 :',len(embedding_dict['respectable']))

[-0.049773   0.19903    0.10585    0.1391    -0.32395    0.44053
  0.3947    -0.22805   -0.25793    0.49768    0.15384   -0.08831
  0.0782    -0.8299    -0.037788   0.16772   -0.45197   -0.17085
  0.74756    0.98256    0.81872    0.28507    0.16178   -0.48626
 -0.006265  -0.92469   -0.30625   -0.067318  -0.046762  -0.76291
 -0.0025264 -0.018795   0.12882   -0.52457    0.3586     0.43119
 -0.89477   -0.057421  -0.53724    0.25587    0.55195    0.44698
 -0.24252    0.29946    0.25776   -0.8717     0.68426   -0.05688
 -0.1848    -0.59352   -0.11227   -0.57692   -0.013593   0.18488
 -0.32507   -0.90171    0.17672    0.075601   0.54896   -0.21488
 -0.54018   -0.45882   -0.79536    0.26331    0.18879   -0.16363
  0.3975     0.1099     0.1164    -0.083499   0.50159    0.35802
  0.25677    0.088546   0.42108    0.28674   -0.71285   -0.82915
  0.15297   -0.82712    0.022112   1.067     -0.31776    0.1211
 -0.069755  -0.61327    0.27308   -0.42638   -0.085084  -0.17694
 -0.0090944  0.1109     0.

In [26]:
embedding_matrix = np.zeros((vocab_size, 100))
print('임베딩 행렬의 크기(shape) :',np.shape(embedding_matrix))

임베딩 행렬의 크기(shape) : (16, 100)


In [28]:
print(tkz.word_index.items())

dict_items([('nice', 1), ('great', 2), ('best', 3), ('amazing', 4), ('stop', 5), ('lies', 6), ('pitiful', 7), ('nerd', 8), ('excellent', 9), ('work', 10), ('supreme', 11), ('quality', 12), ('bad', 13), ('highly', 14), ('respectable', 15)])


In [31]:
print('단어 great의 맵핑된 정수 :',tkz.word_index['great'])

단어 great의 맵핑된 정수 : 2


In [32]:
for word, index in tkz.word_index.items():
    vector_value = embedding_dict.get(word)
    if vector_value is not None :
        embedding_matrix[index] = vector_value

In [33]:
embedding_matrix[2]

array([-0.013786  ,  0.38216001,  0.53236002,  0.15261   , -0.29694   ,
       -0.20558   , -0.41846001, -0.58437002, -0.77354997, -0.87866002,
       -0.37858   , -0.18516   , -0.12800001, -0.20584001, -0.22925   ,
       -0.42598999,  0.3725    ,  0.26076999, -1.07019997,  0.62915999,
       -0.091469  ,  0.70348001, -0.4973    , -0.77691001,  0.66044998,
        0.09465   , -0.44893   ,  0.018917  ,  0.33146   , -0.35021999,
       -0.35789001,  0.030313  ,  0.22253001, -0.23236001, -0.19719   ,
       -0.0053125 , -0.25848001,  0.58081001, -0.10705   , -0.17845   ,
       -0.16205999,  0.087086  ,  0.63028997, -0.76648998,  0.51618999,
        0.14072999,  1.01900005, -0.43136001,  0.46138   , -0.43584999,
       -0.47567999,  0.19226   ,  0.36065   ,  0.78987002,  0.088945  ,
       -2.78139997, -0.15366   ,  0.01015   ,  1.17980003,  0.15167999,
       -0.050112  ,  1.26259995, -0.77526999,  0.36030999,  0.95761001,
       -0.11385   ,  0.28035   , -0.02591   ,  0.31246001, -0.15

In [35]:
output_dim = 100

glove_model = Sequential()
e = Embedding(vocab_size, output_dim, weights=[embedding_matrix], input_length = max_len)
glove_model.add(e)
glove_model.add(Flatten())
glove_model.add(Dense(1, activation='sigmoid'))
glove_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
model.fit(x_train, y_train, epochs=100, verbose=2)

Epoch 1/100
1/1 - 0s - loss: 0.5154 - acc: 1.0000 - 6ms/epoch - 6ms/step
Epoch 2/100
1/1 - 0s - loss: 0.5133 - acc: 1.0000 - 4ms/epoch - 4ms/step
Epoch 3/100
1/1 - 0s - loss: 0.5113 - acc: 1.0000 - 2ms/epoch - 2ms/step
Epoch 4/100
1/1 - 0s - loss: 0.5092 - acc: 1.0000 - 0s/epoch - 0s/step
Epoch 5/100
1/1 - 0s - loss: 0.5071 - acc: 1.0000 - 13ms/epoch - 13ms/step
Epoch 6/100
1/1 - 0s - loss: 0.5051 - acc: 1.0000 - 2ms/epoch - 2ms/step
Epoch 7/100
1/1 - 0s - loss: 0.5030 - acc: 1.0000 - 0s/epoch - 0s/step
Epoch 8/100
1/1 - 0s - loss: 0.5009 - acc: 1.0000 - 13ms/epoch - 13ms/step
Epoch 9/100
1/1 - 0s - loss: 0.4988 - acc: 1.0000 - 2ms/epoch - 2ms/step
Epoch 10/100
1/1 - 0s - loss: 0.4967 - acc: 1.0000 - 0s/epoch - 0s/step
Epoch 11/100
1/1 - 0s - loss: 0.4946 - acc: 1.0000 - 0s/epoch - 0s/step
Epoch 12/100
1/1 - 0s - loss: 0.4926 - acc: 1.0000 - 2ms/epoch - 2ms/step
Epoch 13/100
1/1 - 0s - loss: 0.4905 - acc: 1.0000 - 0s/epoch - 0s/step
Epoch 14/100
1/1 - 0s - loss: 0.4884 - acc: 1.0000 - 

In [37]:
import gensim

!pip install gdown
!gdown https://drive.google.com/uc?id=1Av37IVBQAAntSe1X3MOAl5gvowQzd2_j

word2vec_model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin.gz', binary=True)
print('모델의 크기(shape) :',word2vec_model.vectors.shape) # 모델의 크기 확인

Downloading...
From (original): https://drive.google.com/uc?id=1Av37IVBQAAntSe1X3MOAl5gvowQzd2_j
From (redirected): https://drive.google.com/uc?id=1Av37IVBQAAntSe1X3MOAl5gvowQzd2_j&confirm=t&uuid=3f752049-8ea3-46aa-8aa9-c4fcd33a69f8
To: C:\Users\USER\Desktop\project\ML_study\GoogleNews-vectors-negative300.bin.gz

  0%|          | 0.00/1.65G [00:00<?, ?B/s]
  0%|          | 524k/1.65G [00:00<23:24, 1.17MB/s]
  0%|          | 1.05M/1.65G [00:00<15:26, 1.78MB/s]
  0%|          | 1.57M/1.65G [00:00<12:08, 2.26MB/s]
  0%|          | 2.10M/1.65G [00:00<10:20, 2.65MB/s]
  0%|          | 2.62M/1.65G [00:01<09:31, 2.88MB/s]
  0%|          | 3.15M/1.65G [00:01<09:04, 3.02MB/s]
  0%|          | 3.67M/1.65G [00:01<08:34, 3.19MB/s]
  0%|          | 4.19M/1.65G [00:01<09:18, 2.94MB/s]
  0%|          | 4.72M/1.65G [00:01<10:15, 2.67MB/s]
  0%|          | 5.24M/1.65G [00:01<09:38, 2.84MB/s]
  0%|          | 5.77M/1.65G [00:02<08:39, 3.16MB/s]
  0%|          | 6.29M/1.65G [00:02<08:02, 3.40MB/s]
  0%| 

모델의 크기(shape) : (3000000, 300)


In [39]:
embedding_matrix = np.zeros((vocab_size, 300))
print('임베딩 행렬의 크기(shape) :',np.shape(embedding_matrix))

임베딩 행렬의 크기(shape) : (16, 300)


In [40]:
def get_vector(word) :
    if word in word2vec_model:
        return word2vec_model[word]
    else :
        return None

In [41]:
for word, index in tkz.word_index.items():
    vector_value = get_vector(word)
    if vector_value is not None:
        embedding_matrix[index] = vector_value

In [43]:
print('단어 nice의 맵핑된 정수 :', tkz.word_index['nice'])

단어 nice의 맵핑된 정수 : 1


In [47]:
model = Sequential()
model.add(Input(shape=(max_len,), dtype='int32'))
e = Embedding(vocab_size, 300, weights=[embedding_matrix], input_length=max_len, trainable=False)
model.add(e)
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
model.fit(x_train, y_train, epochs=100, verbose=2)

Epoch 1/100
1/1 - 0s - loss: 0.7293 - acc: 0.2857 - 389ms/epoch - 389ms/step
Epoch 2/100
1/1 - 0s - loss: 0.7106 - acc: 0.2857 - 8ms/epoch - 8ms/step
Epoch 3/100
1/1 - 0s - loss: 0.6923 - acc: 0.2857 - 4ms/epoch - 4ms/step
Epoch 4/100
1/1 - 0s - loss: 0.6746 - acc: 0.2857 - 0s/epoch - 0s/step
Epoch 5/100
1/1 - 0s - loss: 0.6575 - acc: 0.5714 - 0s/epoch - 0s/step
Epoch 6/100
1/1 - 0s - loss: 0.6408 - acc: 0.7143 - 16ms/epoch - 16ms/step
Epoch 7/100
1/1 - 0s - loss: 0.6247 - acc: 0.8571 - 0s/epoch - 0s/step
Epoch 8/100
1/1 - 0s - loss: 0.6091 - acc: 1.0000 - 0s/epoch - 0s/step
Epoch 9/100
1/1 - 0s - loss: 0.5940 - acc: 1.0000 - 16ms/epoch - 16ms/step
Epoch 10/100
1/1 - 0s - loss: 0.5793 - acc: 1.0000 - 0s/epoch - 0s/step
Epoch 11/100
1/1 - 0s - loss: 0.5652 - acc: 1.0000 - 0s/epoch - 0s/step
Epoch 12/100
1/1 - 0s - loss: 0.5514 - acc: 1.0000 - 16ms/epoch - 16ms/step
Epoch 13/100
1/1 - 0s - loss: 0.5382 - acc: 1.0000 - 187us/epoch - 187us/step
Epoch 14/100
1/1 - 0s - loss: 0.5253 - acc: 1